In [1]:
import numpy as np 
import numpy.linalg as LA
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/daniel_vander-hyde/Documents/git/playground/cust_python')
import optimoe.tr_mat as opt
from optimoe.tr_mat import *
import scipy.io as sio
import scipy.signal as sig

In [ ]:
###########################################################################
# Y - arm 
###########################################################################

# Lengths from [1, 2, 7]. Positiosn of ITMY AR and CP1 X and Y from [7].
# Refl, Trans, loss, masses [kg], RoCs [m] from [3].
# ITM Thickness from [8], averaged over the 4 values.
# CP03 Thickness from [9], averaged over the 4 values.

    # Checking if there is a compensation plate
    isCP = False
    if 'CPY1' in kat.components or 'CPYar1' in kat.components:
        isCP = True

    # CPY (CP 03). No CP in [1])
    # -------------
    # Thickness
    th_cpy = 0.10001                  
    # Loss
    L_cpy1 = 0.55e-6
    L_cpy2 = 0.57e-6
    # Refl
    R_cpy1 = 67e-6
    R_cpy2 = 15e-6
    # Trans
    T_cpy1 = 1.0 - R_cpy1 - L_cpy1
    T_cpy2 = 1.0 - R_cpy2 - L_cpy2

    # Input test mass Y (ITM 08)
    # -------------

    # Thermal and substrate lenses in ITMY. From other kat-file. Check!
    TLY_f = 34.5e3           # Thermal lens ITMY, focal length
    SLY_f = np.inf           # Constant ITMY substrate lens, focal length
    # Refl, Trans, and loss.
    #  - HR
    T_itmy_hr = 0.0148                     # 1.48 ppm in [2] (missing %?).
    L_itmy_hr = (14.0 + 0.3)*1e-6          # Scatter + absorbtion. [1] has modified
                                           # this to 65 ppm to get correct PRC gain. 
    R_itmy_hr = 1.0-T_itmy_hr-L_itmy_hr    # = 0.98518570
    #  - AR
    R_itmy_ar = 250e-6                     # 25 ppm in [1]
    L_itmy_ar = 0.6e-6                     # Absorbtion. Not set in [1].
    T_itmy_ar = 1.0-R_itmy_ar-L_itmy_ar    # = 0.9997494
    # Thickness
    th_itmy = 0.19929                      # Not set in [1].
    # Radius of curvature
    RoC_itmy_hr = -1940.7
    # Mass
    M_itmy = 39.420                        # [1] using 40 kg.    

    # End test mass Y (ETM 09)
    # -------------

    # Refl, Trans, and loss.
    #  - HR
    T_etmy_hr = 3.6e-6
    L_etmy_hr = (9.0 + 0.3)*1e-6           # Scatter + absorbtion. [1] has modified
                                           # this to 30  ppm to correct cavity gains.
    R_etmy_hr = 1.0-T_etmy_hr-L_etmy_hr    # = 0.9999871
    #  - AR
    R_etmy_ar = 230e-6                     # 23 ppm in [1].
    L_etmy_ar = 0                          # Unknown
    T_etmy_ar = 1.0-R_etmy_ar-L_etmy_ar    # = 0.99977
    # Thickness
    th_etmy = 0.2    # Estimated from [2], and other fiducial line
                     # measurements. Consistent with other fiels.
    # Radius of curvature
    RoC_etmy_hr = 2242.4
    # Mass
    M_etmy = 39.564                        # [1] using 40 kg.    

    # Lengths [m]
    # -------------
    # BS HR to CPY1 (no CP in [1])
    L_bs_cpy = 4.8470                        
    # CPY2 to ITMY AR (no CP in [1])
    L_cpy_itmyAR = 0.0200                     
    # Optical distance from BS HR to ITMY AR. 
    L_bs_itmyAR = L_bs_cpy + th_cpy*n_silica + L_cpy_itmyAR  # = 5.0120
    # Arm length. ITMY HR to ETMY HR. 
    L_itmy_etmy = 3994.4850      # From [1, 2], [7] has 3 cm diff in ETMY position.

    # Derived
    # -------------
    # Optical distances
    L_bshr_itmyhr = L_bs_itmyAR + th_itmy*n_silica   # = 5.3009. [1] using 5.3044


    # Set values to kat-object
    # -------------

    # Compensation plate
    if isCP:
        if 'CPY1' in kat.components:
            # R, T, L
            kat.CPY1.R = R_cpy1
            kat.CPY1.T = T_cpy1
            kat.CPY1.L = L_cpy1
            kat.CPY2.R = R_cpy2
            kat.CPY2.T = T_cpy2
            kat.CPY2.L = L_cpy2
            # Thickness
            kat.sCPY.L = th_cpy
        else:
            # R, T, L
            kat.CPYar1.R = R_cpy1
            kat.CPYar1.T = T_cpy1
            kat.CPYar1.L = L_cpy1
            kat.CPYar2.R = R_cpy2
            kat.CPYar2.T = T_cpy2
            kat.CPYar2.L = L_cpy2
            # Thickness
            kat.sCPY.L = th_cpy

    # Thermal and substrate lenses
    kat.TLY_f.value = TLY_f
    kat.SLY_f.value = SLY_f

    # ITMY
    kat.ITMY.R = R_itmy_hr
    kat.ITMY.T = T_itmy_hr
    kat.ITMY.L = L_itmy_hr
    # Putting Refl on loss for AR due to wedge.
    kat.ITMYAR.R = 0
    kat.ITMYAR.T = T_itmy_ar
    kat.ITMYAR.L = L_itmy_ar + R_itmy_ar
    # Thickness
    kat.ITMYsub.L = th_itmy
    # RoC
    kat.ITMY.Rc = RoC_itmy_hr
    # Mass
    kat.ITMY.mass = M_itmy

    # ETMY
    kat.ETMY.R = R_etmy_hr
    kat.ETMY.T = T_etmy_hr
    kat.ETMY.L = L_etmy_hr
    # Putting Refl on loss for AR due to wedge.
    kat.ETMYAR.R = 0
    kat.ETMYAR.T = T_etmy_ar
    kat.ETMYAR.L = L_etmy_ar + R_etmy_ar
    # Thickness
    kat.ETMYsub.L = th_etmy
    # RoC
    kat.ETMY.Rc = RoC_etmy_hr
    # Mass
    kat.ETMY.mass = M_etmy

    # Lengths
    if isCP:
        kat.lBS2CPY.L = L_bs_cpy    
        kat.lCPY2ITMY.L = L_cpy_itmy
    else:
        kat.ly1.L = L_bs_itmyAR
    kat.LY.L = L_itmy_etmy

In [ ]:
###########################################################################
    # X - arm
    ###########################################################################


    # CPX (CP 06), no CP used in [1].
    # -------
    R_cpx1 = 33e-6
    L_cpx1 = 0.6e-6
    T_cpx1 = 1.0-R_cpx1-L_cpx1

    R_cpx2 = 8e-6
    L_cpx2 = 0.6e-6
    T_cpx2 = 1.0-R_cpx2-L_cpx2
    # Thickness, average from [10]
    th_cpx = 0.10011

    # ITMX (ITM 04)
    # --------
    # Refl, Trans, and loss
    T_itmx_hr = 0.0148                    # 1.48 ppm in [2] (missing %?)
    L_itmx_hr = (10.0 + 0.4)*1e-6         # [1] using 65 ppm, adjusted for cavity gains.
    R_itmx_hr = 1.0-T_itmx_hr-L_itmx_hr   # = 0.9851896

    R_itmx_ar = 164e-6                    # [1] using 16.4 ppm
    L_itmx_ar = 0.5e-6                    # Not set in [1]
    T_itmx_ar = 1.0-R_itmx_ar-L_itmx_ar   # = 0.9998355
    #  Thickness, average from [11]
    th_itmx = 0.19996                     # Not set in [1]
    # Radius of curvature
    RoC_itmx_hr = -1937.9
    # Mass
    M_itmx = 39.603                       # [1] using 40 kg      

    # Thermal and substrate lenses in ITMX. From other kat-file. Check!
    TLX_f = 34.5e3           # Thermal lens ITMX, focal length
    SLX_f = np.inf           # Constant ITMX substrate lens, focal length

    # ETMX (ETM 07)
    # -----
    # Refl, trans, and loss
    T_etmx_hr = 3.7e-6                     
    L_etmx_hr = (10.6 + 0.3)*1e-6          # Scatter + absorbtion. [1] using 30 ppm for cavity gains. 
    R_etmx_hr = 1.0-L_etmx_hr-T_etmx_hr    # = 0.9999854

    R_etmx_ar = 200e-6                     # [1] using 20 ppm. 
    L_etmx_ar = 0                          # Unspecified.
    T_etmx_ar = 1.0-R_etmx_ar-L_etmx_ar    # = 0.9998
    # Thickness, average from [12]
    th_etmx = 0.19992                      # Not set in [1]
    # Radius of curvature
    RoC_etmx_hr = 2239.7
    # Mass
    M_etmx = 39.620                       # [1] using 40 kg      

    # Lengths
    # --------------

    # Position of BSar from [2], position of CPX1 from [2,7].
    L_bs_cpx = 4.8295
    # Position of CPX2 from [2,7]. Position of ITMX AR from [7]. Consistent with [2]. 
    L_cpx_itmx = 0.0200
    # Positions from [2].
    L_itmx_etmx = 3994.4850

    # Derived
    # -------------
    # Optical distance from BS AR to ITMX AR. 
    L_bsar_itmxar = L_bs_cpx + th_cpx*n_silica + L_cpx_itmx  # = 4.9946
    # Optical distances from BS HR to ITMY HR
    L_bshr_itmxhr = L_bsar_itmxar + (th_itmx + th_bs_beam)*n_silica # = 5.3843. [1] using 5.3844

    # Set values to kat-object
    # -------------

    # Thermal and substrate lenses
    kat.TLX_f.value = TLX_f
    kat.SLX_f.value = SLX_f

    # Lengths
    if isCP:
        # BSARX to CPX1   
        kat.lBS2CPX.L = L_bs_cpx
        # CPX2 to ITMXAR 
        kat.lCPX2ITMX.L = L_cpx_itmx
    else:
        kat.lx1.L = L_bsar_itmxar
    # ITMXHR to ETMXHR (arm length)
    kat.LX.L = L_itmx_etmx

    # ITMX
    kat.ITMX.R = R_itmx_hr
    kat.ITMX.T = T_itmx_hr
    kat.ITMX.L = L_itmx_hr
    # Putting Refl on loss for AR due to wedge.
    kat.ITMXAR.R = 0
    kat.ITMXAR.T = T_itmx_ar
    kat.ITMXAR.L = L_itmx_ar + R_itmx_ar
    # Thickness
    kat.ITMXsub.L = th_itmx
    # RoC
    kat.ITMX.Rc = RoC_itmx_hr

    # ETMX
    kat.ETMX.R = R_etmx_hr
    kat.ETMX.T = T_etmx_hr
    kat.ETMX.L = L_etmx_hr
    # Putting Refl on loss for AR due to wedge.
    kat.ETMXAR.R = 0
    kat.ETMXAR.T = T_etmx_ar
    kat.ETMXAR.L = L_etmx_ar + R_etmx_ar
    # Thickness
    kat.ETMXsub.L = th_etmx
    # RoC
    kat.ETMX.Rc = RoC_etmx_hr
    # Refl, trans and loss

    # Compensation plate
    if isCP:
        # Refl, Trans, Loss
        if 'CPX1' in kat.components:
            kat.CPX1.R = R_cpx1
            kat.CPX1.T = T_cpx1
            kat.CPX1.L = L_cpx1
            kat.CPX2.R = R_cpx2
            kat.CPX2.T = T_cpx2
            kat.CPX2.L = L_cpx2
        else:
            kat.CPXar1.R = R_cpx1
            kat.CPXar1.T = T_cpx1
            kat.CPXar1.L = L_cpx1
            kat.CPXar2.R = R_cpx2
            kat.CPXar2.T = T_cpx2
            kat.CPXar2.L = L_cpx2
        # Thickness
        kat.sCPX.L = th_cpx



In [48]:
RoC_itmx_hr = -1937.9
RoC_etmx_hr = 2239.7
L_itmx_etmx = 3994.4850

In [40]:
RoC_itmy_hr = -1940.7
RoC_etmy_hr = 2242.4
L_itmy_etmy = 3994.4850

### RTGP XARM

In [50]:
cmirror_ITMX = opt.cmirror(RoC_itmx_hr)
cmirror_ETMX = opt.cmirror(RoC_etmx_hr)
LX = opt.space(L_itmx_etmx)
RTMx = cmirror_ITMX*LX*cmirror_ETMX*LX
gpx = opt.rt_gouy(RTMx)

In [ ]:
cmirror_ITMY = opt.cmirror(RoC_itmy_hr)
cmirror_ETMY = opt.cmirror(RoC_etmy_hr)

